In [3]:
from query_representation.query import *

qfn = "../ceb-imdb-full/1a/1a1.pkl"
qrep = load_qrep(qfn)

# extract basic properties of the query representation format

print("""Query has {} tables, {} joins, {} subplans.""".format(
    len(qrep["join_graph"].nodes()), len(qrep["join_graph"].edges()),
    len(qrep["subset_graph"].nodes())))

tables, aliases = get_tables(qrep)

print("Tables: ")
for i,table in enumerate(tables):
    print(table, aliases[i])

print("Joins: ")
joins = get_joins(qrep)
for join in joins:
    print(join)
# print(("\n").join(joins))

# preds, pred_cols, pred_types, pred_vals = get_predicates(qrep)
# print("Predicates: ")
# for i in range(len(preds)):
#     for j, pred in enumerate(preds[i]):
#         print(pred.strip(" "))
#         print("     Predicate column: ", pred_cols[i][j])
#         print("       Predicate type: ", pred_types[i][j])
#         print("     Predicate values: ", pred_vals[i][j])

Query has 9 tables, 9 joins, 107 subplans.
Tables: 
title t
cast_info ci
movie_info mi1
movie_info mi2
info_type it1
info_type it2
kind_type kt
name n
role_type rt
Joins: 
t.id = ci.movie_id
t.id = mi1.movie_id
t.id = mi2.movie_id
t.kind_id = kt.id
ci.person_id = n.id
ci.role_id = rt.id
mi1.movie_id = mi2.movie_id
mi1.info_type_id = it1.id
mi2.info_type_id = it2.id


In [7]:
from query_representation.query import *

qfn = "../ceb-imdb-full/1a/1a1.pkl"
qrep = load_qrep(qfn)

# for getting cardinality estimates of every subplan in the query
ests = get_postgres_cardinalities(qrep)
trues = get_true_cardinalities(qrep)

for k,v in ests.items():
    print("Subplan, joining tables: ", k)
    subsql = subplan_to_sql(qrep, k)
    print("Subplan SQL: ", subsql)
    print("   True cardinality: ", trues[k])
    print("PostgreSQL estimate: ", v)
    print("****************")

Subplan, joining tables:  ('ci', 't')
Subplan SQL:  SELECT COUNT(*) FROM cast_info AS ci , title AS t WHERE  1950 < t.production_year AND  t.production_year <= 1970 AND t.id = ci.movie_id
   True cardinality:  2632262
PostgreSQL estimate:  1279393
****************
Subplan, joining tables:  ('mi1', 't')
Subplan SQL:  SELECT COUNT(*) FROM movie_info AS mi1 , title AS t WHERE  1950 < t.production_year AND  mi1.info IN ('Action','Adventure','Documentary','Romance','Short') AND  t.production_year <= 1970 AND t.id = mi1.movie_id
   True cardinality:  42373
PostgreSQL estimate:  22050
****************
Subplan, joining tables:  ('mi2', 't')
Subplan SQL:  SELECT COUNT(*) FROM movie_info AS mi2 , title AS t WHERE  1950 < t.production_year AND  mi2.info IN ('Black and White','Color') AND  t.production_year <= 1970 AND t.id = mi2.movie_id
   True cardinality:  138028
PostgreSQL estimate:  43045
****************
Subplan, joining tables:  ('kt', 't')
Subplan SQL:  SELECT COUNT(*) FROM kind_type AS k